In [2]:
!pip install mne

In [15]:
import os
from google.colab import drive

mount_point = '/content/drive'
drive.mount(mount_point, force_remount=True)

file_path = "/content/drive/MyDrive/eeg-motor-movementimagery-dataset-1.0.0.zip"

if os.path.exists(file_path):
    print("File found:", file_path)
else:
    print("File NOT found!")


Mounted at /content/drive
File found: /content/drive/MyDrive/eeg-motor-movementimagery-dataset-1.0.0.zip


In [13]:
# EEG Preprocessing Notebook

import numpy as np
import mne
import os
from scipy.signal import butter, filtfilt

In [16]:
import zipfile

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/files")

print("Extracted to /content/eeg_dataset")


Extracted to /content/eeg_dataset


In [20]:
RAW_DATA_DIR = "/content/drive/MyDrive/files/files"       # ← contains S001, S002, ...
OUT_DIR = "/content/drive/MyDrive/files/segments"   # ← new folder for processed segments
os.makedirs(OUT_DIR, exist_ok=True)
print("Subjects found:", os.listdir(RAW_DATA_DIR))

Subjects found: ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S029', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S040', 'S041', 'S042', 'S043', 'S044', 'S045', 'S046', 'S047', 'S048', 'S049', 'S050', 'S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057', 'S058', 'S059', 'S060', 'S061', 'S062', 'S063', 'S064', 'S065', 'S066', 'S067', 'S068', 'S069', 'S070', 'S071', 'S072', 'S073', 'S074', 'S075', 'S076', 'S077', 'S078', 'S079', 'S080', 'S081', 'S082', 'S083', 'S084', 'S085', 'S086', 'S087', 'S088', 'S089', 'S090', 'S091', 'S092', 'S093', 'S094', 'S095', 'S096', 'S097', 'S098', 'S099', 'S100', 'S101', 'S102', 'S103', 'S104', 'S105', 'S106', 'S107', 'S108', 'S109', 'SHA256SUMS.txt', 'RECORDS', '64_channel_sharbrough.pdf', '64_channel_sharbrough.png', 'ANNOTATORS', '64_channe

In [21]:
# Bandpass filter (1–50 Hz)
# -------------------------------------------
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_filter(data, fs, lowcut=1, highcut=50):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return filtfilt(b, a, data, axis=1)
# Segment EEG into fixed windows
# -------------------------------------------
def segment_eeg(data, segment_length, fs):
    samples = segment_length * fs
    segments = []

    for start in range(0, data.shape[1] - samples + 1, samples):
        seg = data[:, start:start+samples]
        segments.append(seg)

    return segments

In [ ]:
# Preprocess all subjects S001–S109
# ===========================================
fs = 160           # PhysioNet sampling rate
segment_sec = 2    # 2-second windows

for subj in sorted(os.listdir(RAW_DATA_DIR)):
    subj_path = os.path.join(RAW_DATA_DIR, subj)

    if not subj.startswith("S"):
        continue

    print(f"\nProcessing subject: {subj}")
    files = os.listdir(subj_path)

    for file in files:
        if file.endswith(".edf"):
            fpath = os.path.join(subj_path, file)
            print("  Loading:", fpath)

            raw = mne.io.read_raw_edf(fpath, preload=True, verbose=False)
            eeg = raw.get_data()             # shape: (channels, samples)

            # Filter
            eeg_filtered = apply_filter(eeg, fs)

            # Segment
            segments = segment_eeg(eeg_filtered, segment_sec, fs)

            # Save segments
            base = file.replace(".edf", "")
            for i, seg in enumerate(segments):
                out_name = f"{subj}_{base}_seg{i}.npy"
                out_path = os.path.join(OUT_DIR, out_name)
                np.save(out_path, seg)

print("\n Preprocessing complete.")
print("Segments saved to:", OUT_DIR)


Processing subject: S001
  Loading: /content/drive/MyDrive/files/files/S001/S001R08.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R05.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R01.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R10.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R11.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R03.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R07.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R04.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R06.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R13.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R12.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R14.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R02.edf
  Loading: /content/drive/MyDrive/files/files/S001/S001R09.edf

Processing subject: S002
  Loading: /content/drive/MyDrive/files/files/S002/S002R11.edf
  L